In [1]:
import torch


In [2]:
torch.__version__

'1.0.0.dev20181018'

In [3]:
import torch
import torch.utils.data
import torchvision
import pandas as pd
import torchvision.transforms as transforms
import numpy as np
import os
import glob
torch.backends.cudnn.deterministic = True
import pandas as pd
import os
import PIL
from PIL import Image

#import Levenshtein

In [4]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import models, transforms,datasets
from torch.autograd import Variable
import torch.nn as nn
import string

## For csv type of loading

In [49]:
## NOvenmber 23
import string
char_set_aadhar = string.ascii_letters + ' -/:' + string.digits    
map_chars_aadhar = {x:i+1 for i,x in enumerate(set(char_set_aadhar))}
map_chars_aadhar['_'] = 0
reverse_map_aadhar = {map_chars_aadhar[y]:y for y in map_chars_aadhar.keys()}

In [88]:
import pandas as pd
import string
class images_random_text_dataset(torch.utils.data.Dataset):
    """
    A customized data loader.
    """
    

    

# ON the assumption that we give a csv file which has 3 features path,name,wordlength respectively so that they could be acc
#        accessed directly through a dataframe and passed to __getitem__ function in dataset
        
    def __init__(self, dataset_filtered,map_chars,reverse_map,width_=128):
        """ Intialize the dataset
        """
        self.dataset = dataset_filtered
        #self.filenames = list(self.dataset.path)
        #self.root = root_
        self.imgWidth = width_
        self.map_chars = map_chars
        self.reverse_map = reverse_map
        number_of_files = len(self.dataset)
        self.root_dir = root_dir
        self.number_of_files = len(self.dataset)

        
    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        ## CSV file Column Names
        dataset = self.dataset
        #print(dataset.head())
        image_path_list = list(dataset.path)
        #print(len(image_path_list))
        #print('image_path_list type :'+ str(type(image_path_list)))
        data_wordlength = list(dataset.wordlength)
        data_name_list = list(dataset.name)
        ##############
        #relative_path = image_path_list[index]
        #full_path = 
        ################
        #print(index)
        image = Image.open(image_path_list[index])
        
        orig_name_,name_length = data_name_list[index],data_wordlength[index]
        ## USED LOWER BECAUSE THE DICTIONARY IS BUILT ONLY FOR LOWER CASE
        name_ = orig_name_.lower() 

        y_name,y_name_length = self.get_names_as_number_arrays(name_)
        if image.size[0] >self.imgWidth:
            size_reduce = transforms.Resize((32,self.imgWidth))
            image = size_reduce(image)
        else:
            height = image.size[1]
            width = image.size[0]
            size_pad = transforms.Pad((0,0,self.imgWidth-width,32-height), fill=(255,255,255), padding_mode='constant')
            image = size_pad(image)
        self.transform = transforms.ToTensor()
        input_image = self.transform(image)

        return (input_image,y_name,y_name_length)

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.number_of_files
    def get_names_as_number_arrays(self,name_from_df):
        targe1 = []
        tar_length = []
        max_len = 0
        name_from_df = str(name_from_df)
        count_ = len(name_from_df)
        #print(image_path)
        #for path in image_paths:
        name_ = name_from_df
        #print(name_)
        name_encoded = self.alphabet_encode(name_)     ## ENcode the name to list of numbers 'a-z' & ' '
        #targe1.extend(name_encoded)    ##EXTEND IS USED WITH CUDA
        #targe.append(name_)           ## CAN BE USED ONLY WITH "CPU"
        name_len = len(name_encoded)

        return (name_encoded,name_len)

    def alphabet_encode(self,text):

    
        nums = [self.map_chars.get(x,0) for x in text]

        return nums
    def alphabet_decode(self,num_list):
       
        name = [self.reverse_map.get(x) for x in num_list]

        return ''.join(name)

In [6]:
def get_images_with_wordlength_less_than(df,word_len):
    
    word_length_less_than = df.wordlength < word_len
    dataset_filtered_l = df[word_length_less_than]
    #dataset_path_filtered = absolute_to_relative_path(dd_filtered,directory_path)
    
    return dataset_filtered_l
def get_images_with_image_width_less_than(df,img_width):
    
    image_less_than = df.image_width < img_width
    dataset_filtered_w = df[image_less_than]
    #dataset_path_filtered = absolute_to_relative_path(dd_filtered,directory_path)
    
    return dataset_filtered_w

#### Input to Data_set_creation

In [7]:
import torch
import torchvision
import pandas as pd
import torchvision.transforms as transforms
import numpy as np
import os
import glob
torch.backends.cudnn.deterministic = True
import os
import PIL
from PIL import Image
#import Levenshtein
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import models, transforms,datasets
from torch.autograd import Variable
import torch.nn as nn


In [8]:
class Unit(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,stride):
        super().__init__()
        
        
        self.conv = nn.Conv2d(in_channels=in_channels, kernel_size=3, out_channels=out_channels, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.relu = nn.ReLU(inplace=True)

In [9]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
def conv7x7(in_channels, out_channels, stride=2):
    return nn.Conv2d(in_channels, out_channels, kernel_size=7, stride=stride, padding=0, bias=False)

In [10]:


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        print('bn1 type' + str(type(self.bn1)))
        self.relu = nn.ReLU(inplace=True)
        #self.part1 = Unit(in_channels=in_channels,kernel_size=3,out_channels=out_channels,stride=stride)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [11]:
class ResBlock(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv = conv7x7(3, 64)
        self.bn = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.max_pool = nn.MaxPool2d(kernel_size=3,stride=2)
        self.layer1 = self.make_layer(block, 64, 1)
        self.layer2 = self.make_layer(block, 128, 1)
        self.layer3 = self.make_layer(block, 256, 1)
        self.layer4 = self.make_layer(block, 512, 1)
        self.lastConvlayer = conv3x3(512,64)
        self.linearLayer = nn.Linear(384,40)
        self.probLayer = nn.LogSoftmax(2) #dimension along which log_softmax to be calculated.
        #sequence the columns in all channels into a sngle column so you have (c*h,w) dmensional matrix
        
        
        #class_matrix = self.linearLayer(seq_matrix)
       
        #prob_matrix = self.probLayer(class_matrix)

        #self.avg_pool = nn.AvgPool2d(8)
        #self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
#         print('Input Size ' + str(x.size()))
        out = self.conv(x)
#         print('After Conv layer ' + str(out.size()))
        out = self.bn(out)
#         print('After Batchnorm layer ' + str(out.size()))
        out = self.relu(out)
#         print('After relu layer ' + str(out.size()))
        out = self.max_pool(out)
#         print('After max_pool layer ' + str(out.size()))
        out = self.layer1(out)
#         print('After 1st res block layer ' + str(out.size()))
        out = self.layer2(out)
#         print('After 2nd res block layer ' + str(out.size()))
        out = self.layer3(out)
#         print('After 3rd res block layer ' + str(out.size()))
        out = self.layer4(out)
#         print('After 4th res block layer ' + str(out.size()))
        out = self.lastConvlayer(out)
#         print('After last Conv layer ' + str(out.size()))
        t,channels,rows,cols = out.size()
        out = out.view(t,channels*rows,cols).transpose(1,2) 
        print('After reshaping for linear layer to collapse : ' +str(out.size()) )
        out = self.linearLayer(out)
        print('Linear layer output size :'+ str(out.size()))
       
        prob_matrix = self.probLayer(out)
        print('After last layer ' + str(prob_matrix.size()))
        #dim_mod_for_CTC = prob_matrix.permute(1,0,2)
        #print('input to CTC Loss function'+ str(dim_mod_for_CTC.size()))
        #out = self.avg_pool(out)
        #out = out.view(out.size(0), -1)
        #out = self.fc(out)
        return prob_matrix

In [58]:
def alphabet_encode(text):
#     nums = [0]
#     for x in text.lower():
#         if (x >= 'a' and x <= 'z') or x == ' ':
#             nums.append((ord(x) - 96))
#             nums.append(0)

    nums = [map_chars.get(x,0) for x in text]
    #for i in nums
    #print(nums)
    return nums
def encoded_series_to_list(target_encode_series,target_length_list):
    pos_list = [sum(target_length_list[0:i]) for i in range(0,len(target_length_list)+1)]
    decoded_name = []
    for i in range(len(pos_list)-1):
        #encoded_name.append(list(vv[1][i:i+]))
        #encoded_name.append(list(target_encode_series[pos_list[i]:pos_list[i+1]]))
        encoded_name = list(target_encode_series[pos_list[i]:pos_list[i+1]])
        decoded_name.append(alphabet_decode_single(encoded_name))
    return decoded_name
def alphabet_decode_single(num_list):

    name = [reverse_map.get(x) for x in num_list]


    #name = [chr(int(nu + 96)) if 0<nu<27 else (nu not 0) str(nu-26) for nu in num_list]
    return ''.join(name)

In [92]:
ctc_loss = nn.CTCLoss()
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx,(inputs, target_name_encoded, target_lengths) in enumerate(train_loader):
        #print('labels')
        #print(labels)
        #print('shape')
        #print(inputs.shape)
        
        #target = get_names_as_list()
        input_batch_size_ = len(inputs)
        image_batch =torch.stack(inputs) 
        #encoded_target_train, target_lengths = get_names_as_number_arrays(images_paths)
        #print('encoded_target_train')
        #print(encoded_target_train)
        #print('target_lengths')
        #print(target_lengths)
        #print(paths)
        ##print('encoded target : ')
        #print(encoded_target_train)
        #print(encoded_target_train)
        encoded_target_train = torch.LongTensor(target_name_encoded)
        #encoded_target_train = torch.LongTensor(encoded_target_train)
        optimizer.zero_grad()
        #print(encoded_target_train)
        image_batch = image_batch.to(device)
        print(image_batch.shape)
        prob_sequence_matrix = model(image_batch)
        #prob_sequence_matrix.
        prob_sequence_matrix = prob_sequence_matrix.permute(1,0,2)
        ###Normally for a 128 width image input length is it is 30 for the model
        in_length_tensor = np.full((1,int(input_batch_size_)),prob_sequence_matrix.shape[0])
        in_length_tensor = torch.LongTensor(in_length_tensor)
        target_length_tensor = torch.LongTensor(target_lengths)
        #loss = F.nll_loss(output, target)
        in_length_tensor = in_length_tensor.to(device)
        target_length_tensor = target_length_tensor.to(device)
        encoded_target_train = encoded_target_train.to(device)
#         if batch_idx % 5 == 0:
#             for param in model.parameters():
#                 print('Param : ' + str(param))
#                 print('Param Grad :' + str(param.grad))
        #print('input to CTC Loss function'+ str(dim_mod_for_CTC.size()))
#         num_mat = []
#         for images in range(prob_sequence_matrix.shape[0]):
#             numli = []
#             for cols in range(prob_sequence_matrix[image].shape[0]):
#                 numli.append(np.argmax(prob_sequence_matrix[images][cols].detach().numpy()))
#             num_mat.append(numli)
        #print('prob_sequence_matrix')
        #print(prob_sequence_matrix)
#         print('prob_sequence_matrix shape :' + str(prob_sequence_matrix.shape))
        #print('encoded_target_train')
        #print(encoded_target_train)
#         print('encoded_target_train shape :' + str(encoded_target_train.shape))
#        print('in_length_tensor' + str(in_length_tensor.shape))
#        print('target_length_tensor'+ str(target_length_tensor.shape))
        
        loss = ctc_loss(prob_sequence_matrix, encoded_target_train, in_length_tensor, target_length_tensor)
        writer.add_scalar('train_loss', loss, epoch)  ## To Display in TensorboardX
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(),1)
        torch.nn.utils.clip_grad_value_(model.parameters(),1)
        optimizer.step()
        #if batch_idx % args.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(inputs), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [309]:
!pip install python-levenshtein

  Could not find a version that satisfies the requirement python (from versions: )
No matching distribution found for python
You are using pip version 8.1.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [60]:
def edit_dist_calc_list(pred_word_list,targ_word_list):
    dist = 0
    test_batch_size = len(pred_word_list)
    #max_len = len(pred_word_list) if len(pred_word_list)>len(targ_word_list) else len(targ_word_list)
    for i in range(test_batch_size):
        dist += Levenshtein.distance(pred_word_list[i],targ_word_list[i])
    mean_dist = dist/test_batch_size
    return mean_dist

In [61]:
def alphabet_decode(num_list):
    name = [reverse_map.get(x) for x in num_list]


    #name = [chr(int(nu + 96)) if 0<nu<27 else (nu not 0) str(nu-26) for nu in num_list]
    return ''.join(name)
def collapse(names_encoded):
    collapsed_words=[]
    for encoded in names_encoded:
        for i,n in enumerate(encoded):
            if encoded[i] == encoded[i-1]:
                encoded[i-1] = 0
        collapsed_words.append([x for x in encoded if x!=0 ])
    return collapsed_words
def inference(model,image_path):
    
    img = Image.open(image_path)
    ch = preprocess(img)
    #width = img.size[0]
    #height = img.size[1]
    ch = ch.view(1,3,32,128)
    trial_img = ch.to('cuda')
    log_img = model(trial_img)
    print(log_img.shape)
    cpu_img = log_img.to(device = 'cpu')
    final_prob = np.exp(cpu_img.detach().numpy())
    encode_char = np.argmax(final_prob,axis = 2)
    print(encode_char)
    print(encode_char.shape)
    #flattened = encode_char.reshape(encode_char.shape[1])
    #print(flattened)
    #print(flattened.shape)
    collapsed = collapse(encode_char)
    print(collapsed)
    #print(type(collapsed))
    for x in collapsed:
        word = alphabet_decode(x)

    return word

In [87]:
def collapse(names_encoded):
    print(names_encoded)
    collapsed_words=[]
    for encoded in names_encoded:
        for i,n in enumerate(encoded):
            if encoded[i] == encoded[i-1]:
                encoded[i-1] = 0
        collapsed_words.append([x for x in encoded if x!=0 ])
    if(len(collapsed_words) == 0):
        collapsed_words.append(0)
    return collapsed_words

In [13]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for nu,(inputs, target_name_encoded, target_lengths) in enumerate(test_loader):
            print(target_name_encoded)
            #print(type(target_name_encoded))
            print(target_lengths)
            #data, target = data.to(device), target.to(device)
            #images_paths = paths
            input_batch_size_ = len(inputs)
            image_batch =torch.stack(inputs) 
            image_batch = image_batch.to(device)
            pred_mat = model(image_batch)
            #print(pred_mat.shape)
            #print('pred mat' + str(pred_mat.size()))
            encoded_target_test, target_lengths = target_name_encoded,target_lengths
            encoded_target_test = torch.LongTensor(encoded_target_test)
            #test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            #pred = np.argmax(pred_mat.detach().numpy(),axis = 2)
            
#             print('pred')
#             print(pred)
#             print('encoded_target_test')
#             print(encoded_target_test)
            cpu_img = pred_mat.to(device = device)
            final_prob = np.exp(cpu_img.detach().numpy())
            pred_encode_char = np.argmax(final_prob,axis = 2)
            #print(encode_char)
            #print(encode_char.shape)
            #flattened = encode_char.reshape(encode_char.shape[1])
            #print(flattened)
            #print(flattened.shape)
            pred_collapsed = collapse(pred_encode_char)
            #print(pred_collapsed)
            #print(collapsed)
            #pred_words = alphabet_decode_list(pred_collapsed)
            #targ_words = encoded_series_to_list(target_name_encoded,target_lengths)
            print(pred_collapsed)
            #print(targ_words)
            #print(pred_words)
#             pred_mat = pred_mat.permute(1,0,2)
#             in_length_tensor = np.full((1,int(input_batch_size_)),pred_mat.shape[0])
#             in_length_tensor = torch.LongTensor(in_length_tensor)
#             target_length_tensor = torch.LongTensor(target_lengths)
#             #loss = F.nll_loss(output, target)
#             #pred_mat = pred_mat.permute(1,0,2)
#             #cc = ((encoded_target_test - pred).sum(axis=1) == 0).sum()
#             #correct += cc
#             encoded_target_test = encoded_target_test.to(device)
#             in_length_tensor = in_length_tensor.to(device)
#             target_length_tensor = target_length_tensor.to(device)
#             loss = ctc_loss(pred_mat, encoded_target_test, in_length_tensor, target_length_tensor)
            print('Inside testing')
#---------------FOR CPU WITHOUT LEVENSHTEIN DISTSANCE-----------------------------------------
            #distance = edit_dist_calc_list(targ_words,pred_words)
            #writer.add_scalar('distance', distance, epoch)  ## To Display in TensorboardX
            #print(distance)


    #test_loss /= len(test_loader.dataset)
    #print('\nTest set: Average loss: {:.4f}'.format(Avg_test_loss))

In [93]:
import math
# Collate_fn for Dataloader, this is used because the target_length for different length 
def my_collate_concat_label(batch_):
    print('inside my_collate_concat_label')
    print(type(batch_))
    data = [item[0] for item in batch_]
    #data = torch.LongTensor(data)
    target = [item[1] for item in batch_]
    lenn = tuple([item[2] for item in batch_])
    target = np.concatenate(target)
    #lenn = np.concatenate(lenn)
    batch_tuple = (data,target,lenn)
    return batch_tuple
def absolute_to_relative_path(dataset_csv,dir_name):
    path = 'path'
    dataset_csv[path] = [dir_name+('/'.join(x.split('/')[-3:])) for x in dataset_csv[path]]
    return dataset_csv
def change_path(dataset_csv,root_dir_path):    
    dataset_csv['path'] = root_dir_path + dataset_csv['relative_path']
    df_all = dataset_csv
    df_all = df_all.dropna()
    df_all = df_all.reset_index(drop=True)
    return df_all
def warmup(model,full_dataset_as_csv,initial_word_width,W_warm_up_epochs,total_epochs,device):
    l = initial_word_width
    optimizer_w = torch.optim.Adam(model.parameters(), lr = 0.0001)
    alpha_w = 0.00001
    beta_w = 0.001
    delta_w = (np.log10(beta_w)-np.log10(alpha_w))/total_epochs
    #ln_alpha = np.log10(alpha)
    #delta1 = (np.log10(beta-alpha))
    #print(delta)
    log_lr = np.log10(alpha_w)
    for i in range(W_warm_up_epochs):

        ln_lr = log_lr + i*delta_w
        modified_lr = 10**ln_lr
        for param_group in optimizer_w.param_groups:
            param_group['lr'] = modified_lr
        #Generate filtered training data based on length for warmup
        #Filters and returns a Dataframe of the path, name, len as features
        limited_leng = get_images_with_wordlength_less_than(full_dataset_as_csv,l)
        
        ## Map_chars dict is used to encode the chars to number and reverse_map to decode, set the width
        dataset_filtered_length = images_random_text_dataset(limited_leng,map_chars_aadhar,reverse_map_aadhar,width_ = 128)
        print('dataset_filtered_length :' + str(len(dataset_filtered_length)))
        l+=1
        dataset1 = dataset_filtered_length
        train_length = int(len(dataset1)*0.8)
        test_length = len(dataset1) - train_length
        lengths = [train_length,test_length]
        #print(lengths)
        dataset_train,dataset_test = torch.utils.data.random_split(dataset1, lengths)
        data_loader_train = torch.utils.data.DataLoader(dataset_train,batch_size=100,collate_fn=my_collate_concat_label)
        #print(len(data_loader_train))
        data_loader_test = torch.utils.data.DataLoader(dataset_test,batch_size=50,collate_fn=my_collate_concat_label)
        train(device=device,epoch=i+1,model=model,train_loader=data_loader_train,optimizer=optimizer_w)
        torch.save(model.state_dict(),weight_path)
        test(device=device,model=model,test_loader=data_loader_test)
    return model
        
def curriculum_train(model,full_dataset_as_csv,epoch_count,device,initial_width=30,initial_lr = 0.01,decay = 40):
    wid = initial_width
    modified_lr_c = initial_lr
    #t=decay
    optimizer_curr = torch.optim.Adam(model.parameters(), lr = modified_lr_c)
    for i in range(epoch_count):
        #learning_rate = learning_rate*10**(-1*(math.floor((i+3)/t)))
        
        for param_group in optimizer_curr.param_groups:
            param_group['lr'] = modified_lr_c
        modified_lr_c = modified_lr_c*10**(-1*(i/decay))
        print(modified_lr_c)
        #print(width)
        limited_leng = get_images_with_image_width_less_than(full_dataset_as_csv,wid)
        wid += 8
        dataset_filtered_width = images_random_text_dataset(limited_leng,map_chars_aadhar,reverse_map_aadhar,width_ = wid)
        dataset1 = dataset_filtered_width
        
        train_length = int(len(dataset1)*0.8)
        test_length = len(dataset1) - train_length
        lengths = [train_length,test_length]
        dataset_train,dataset_test = torch.utils.data.random_split(dataset1, lengths)
        data_loader_train = torch.utils.data.DataLoader(dataset_train,batch_size=100,collate_fn=my_collate_concat_label)
        data_loader_test = torch.utils.data.DataLoader(dataset_test,batch_size=50,collate_fn=my_collate_concat_label)
        train(device=device,epoch=i+1,model=model,train_loader=data_loader_train,optimizer=optimizer_curr)
        torch.save(model.state_dict(),weight_path)
        test(device=device,model=model,test_loader=data_loader_test)

        
        
        
    
    

In [17]:

start_length_ = 2
W_warm_up_epochs_ = 12
curriculum_epochs_ = 40
warm_up_width_ = 128
initial_word_width_ = 2


In [39]:
for i in range(1,12):
    ln_lr = ln_alpha + i*delta
    print(10**ln_lr)

0.0001148153621496883
0.00013182567385564074
0.00015135612484362088
0.00017378008287493763
0.00019952623149688788
0.00022908676527677723
0.00026302679918953814
0.0003019951720402016
0.0003467368504525317
0.00039810717055349735
0.0004570881896148752


In [30]:
10**-1

0.1

In [73]:
import string
char_set_aadhar = string.ascii_lowercase + ' -/:' + string.digits

map_chars_aadhar = {x:i+1 for i,x in enumerate(set(char_set_aadhar))}
map_chars_aadhar['_'] = 0
reverse_map_aadhar = {map_chars_aadhar[y]:y for y in map_chars_aadhar.keys()}

In [74]:
len(char_set_aadhar)

40

In [94]:
from tensorboardX import SummaryWriter
writer= SummaryWriter('/home/madhevan/Documents/projects/rosetta/tensorboard_files/')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1 = ResBlock(ResidualBlock,[1,1,1,1]).to(device)
weight_path = '/home/madhevan/Documents/projects/rosetta/to_gcp/trial_gcp_rosetta_23.pth'

bn1 type<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
bn1 type<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
bn1 type<class 'torch.nn.modules.batchnorm.BatchNorm2d'>
bn1 type<class 'torch.nn.modules.batchnorm.BatchNorm2d'>


In [95]:
csv_path_name = '/home/madhevan/Documents/projects/rosetta/to_gcp/dataset_details_csv.csv'
data_details = pd.read_csv(csv_path_name,index_col=0)
#data_details = data_details[0:1000]
#root_dir = '/'.join(csv_path_name.split('/')[0:(len(csv_path_name.split('/'))-1)])+'/'
root_dir = '/home/madhevan/Documents/projects/rosetta/data_generation/'

data_with_full_path_csv = change_path(data_details,root_dir)
#model_ = warmup(model1,data_with_full_path_csv,initial_word_width_,W_warm_up_epochs_,42,device = 'cpu')
model_ = warmup(model1,data_with_full_path_csv,initial_word_width_,W_warm_up_epochs_,42,device = 'cpu')
curriculum_train(model1,data_with_full_path_csv,epoch_count=30,device = 'cpu')

dataset_filtered_length :0
dataset_filtered_length :24441
inside my_collate_concat_label
<class 'list'>
torch.Size([100, 3, 32, 128])
After reshaping for linear layer to collapse : torch.Size([100, 30, 384])
Linear layer output size :torch.Size([100, 30, 40])
After last layer torch.Size([100, 30, 40])
Train Epoch: 2 [0/19552 (0%)]	Loss: 48.920475
inside my_collate_concat_label
<class 'list'>
torch.Size([100, 3, 32, 128])
After reshaping for linear layer to collapse : torch.Size([100, 30, 384])
Linear layer output size :torch.Size([100, 30, 40])
After last layer torch.Size([100, 30, 40])
Train Epoch: 2 [100/19552 (1%)]	Loss: 47.080410
inside my_collate_concat_label
<class 'list'>
torch.Size([100, 3, 32, 128])


KeyboardInterrupt: 

dataset_filtered_length :0
[0, 0]
dataset_filtered_length :11
[8, 3]
inside my_collate_concat_label
<class 'list'>
torch.Size([8, 3, 32, 128])
After reshaping for linear layer to collapse : torch.Size([8, 30, 384])
Linear layer output size :torch.Size([8, 30, 40])
After last layer torch.Size([8, 30, 40])
Train Epoch: 2 [0/8 (0%)]	Loss: nan
inside my_collate_concat_label
<class 'list'>
[23 21 15 20 30 40]
(2, 2, 2)
After reshaping for linear layer to collapse : torch.Size([3, 30, 384])
Linear layer output size :torch.Size([3, 30, 40])
After last layer torch.Size([3, 30, 40])
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[], [], []]
Inside testing
dataset_filtered_length :27
[21, 6]
inside my_collate_concat_label
<class 'list'>
torch.Size([21, 3, 32, 128])
After reshaping for linear layer to collapse : torch.Size([21, 30, 384])
Linear layer outp

Train Epoch: 7 [4/104 (50%)]	Loss: nan
inside my_collate_concat_label
<class 'list'>
[40  1 30 23 30  5 21 15 30 13 30  5 34 13 24 40 32 23  3 21 23 30  5 21
  3 21 23 30  5 21 35 20 40  1 30 30  3 40 30 40 32 27  1 30  8  1 30 40
  3 21 23 30  5 21  3 21 23 30  5 21  3 21 23 30  5 21  3 21 23 30  5 21
 39 21 21 17 20 15 23 21 21 30 12  3 21 23 30  5 21 23 30  5 21 23 30 23
 32 35 17 30 12 30 40  3 21 23 30  5 21 23 30  5 21]
(3, 4, 6, 2, 3, 6, 6, 5, 4, 5, 2, 2, 6, 6, 6, 6, 6, 3, 2, 6, 4, 5, 3, 2, 6, 4)
After reshaping for linear layer to collapse : torch.Size([26, 30, 384])
Linear layer output size :torch.Size([26, 30, 40])
After last layer torch.Size([26, 30, 40])
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 

After reshaping for linear layer to collapse : torch.Size([100, 30, 384])
Linear layer output size :torch.Size([100, 30, 40])
After last layer torch.Size([100, 30, 40])
Train Epoch: 10 [0/131 (0%)]	Loss: nan
inside my_collate_concat_label
<class 'list'>
torch.Size([31, 3, 32, 128])
After reshaping for linear layer to collapse : torch.Size([31, 30, 384])
Linear layer output size :torch.Size([31, 30, 40])
After last layer torch.Size([31, 30, 40])
Train Epoch: 10 [31/131 (50%)]	Loss: nan
inside my_collate_concat_label
<class 'list'>
[30 40  1 20 23 30 23 23 30 35  1 21 23 30 35 22 15 23 30  5 21  3 21 23
 30  5 21 17 38 38 35 30 23 17 12 30 35 40  3 21 23 30  5 21 13 24 38 22
 20 22 30 40  5 20 23 30 30 15 23 20 40 23 30  5 21 23 30  5 21 30 12 13
 32 35 22 24  6 23 32 35 21 40 20 17 30 12 17 12 30 34 21 21  3 21 23 30
  5 21 12 30  8  1 40  1 30 23 30  5 21 23 30 23 32 35 30 12 40  1 23 30
  5 21 30 40 12 30 23 15 20 30 40  1 30 23 30  5 21  3 21 23 30  5 21 30
  3 40 30 30 40 40 30 27 3

KeyboardInterrupt: 

Aadhaar Training